In [ ]:
from datetime import datetime, timedelta

file = open("/home/jovyan/trino.secret", "r")
username = 'yj.noh'
password = file.readline().strip()
file.close()

auth = BasicAuthentication(username, password)

conn = connect(
    host='https://trino-auth.emr.ds.woowa.in',
    user = username,
    auth = auth,
    port = 443,
    catalog = 'hive_zeppelin',
    schema = 'sbtasty'
)

cursor = conn.cursor()
date_to = datetime.today().date() - timedelta(days=1)
date_from = "2023-01-01"

sql_query = f"""
    WITH PICK as (
                    select
                        shop_no
                        , shop_name as shop_nm
                        , part_date
                        , case when split_part(address, ' ', 1) LIKE '서울%' then '서울특별시' else split_part(address, ' ', 1) end as rgn1_nm
                        , split_part(address, ' ', 2) as rgn2_nm
                        , 'B1' as service_type
                    from sbstore_hist.shop_master
                    where 1 = 1
                        and part_date >= '{{datefrom}}'
                        and part_date <= '{{dateto}}'
                        and is_baemin_one = 1
                        and is_shop_test = 0
                        and shop_status = 'OPEN'
                union all
                select
                distinct
                    baemin_shop_no as shop_no
                    , name as shop_nm
                    , cast('{{dateTo}}' as varchar) as part_date
                    , split_part(address, ' ', 1) as rgn1_nm
                    , split_part(address, ' ', 2) as rgn2_nm
                    , 'BM' as service_type
                from market.bm_shop
                where 1 = 1
                    and shop_id not in ('142', '105', '151', '141', '153', '104', '109', '196')
                    and baemin_shop_no not in ('0', '02225559985', '10917468', '11', '11111111111', '13035449', '13765225', '222222222', '5555', '7777', '99999998')
                    and is_test_shop = 0
                    and split_part(name, ' ', 1) = 'B마트'
                    and status = 'ACTIVE'
            )
  
    , dlvry as (
        SELECT 

            business_day
            , cast(date_format(reg_date, '%Y-%m-%d') AS DATE) AS reg_date 
            , hour(reg_date) AS hour_reg
            
            , DL.ord_no 
            , DL.dlvry_id 
            , DL.rider_user_id 

            
            , rgn1_nm AS dlvry_rgn1_nm --전달지 rgn1
            , rgn2_nm AS dlvry_rgn2_nm --전달지 rgn2 

            
            , t1.shop_no

            , CASE WHEN substr(DL.ord_no, 1, 1)  = 'T' THEN 'B1'
                WHEN substr(DL.ord_no, 1, 1) != 'T' THEN 'BM'
            END AS service_type
            
        FROM sbtasty.dlvry2 DL
        INNER JOIN sbtasty.rider RI
        ON DL.rider_user_id = RI.rider_user_id
        INNER JOIN  (
                    select service_type
                            , ord_no
                            , split_ord_no
                            , (case when service_type = 'BAERA' THEN ord_no
                                    WHEN service_type = 'BMART' THEN split_ord_no END) AS order_no
                            , part_date
                            , order_status
                            , is_closed
                            , is_test
                            , customer_rgn3_cd 
                            , customer_notified_minute
                            , shop_no
                    FROM delivery_mart.delivery_master 
                    where 1=1
                    AND part_date >= '{{dateFrom}}'
                    AND part_date <= '{{dateTo}}'
                    AND order_status = 'CLOSED'
                    AND is_closed = 1
                    AND is_test = 0
            ) t1
        ON DL.ord_no = t1.order_no
        INNER JOIN sbbi.dim_rgn_cd r 
        ON t1.customer_rgn3_cd = r.rgn3_cd
        WHERE 1 = 1
        AND customer_notified_minute is NOT NULL
        AND business_day >= '{{dateFrom}}'
        AND business_day <= '{{dateTo}}'
        AND agency_id <> 13
        AND pick_up_date IS NOT NULL
        AND RI.rider_type IN ('CROWDSOURCING','INDIVIDUAL_OWNER')

    )

    , dlvry_final as (
        SELECT   
            business_day
            , reg_date
            , hour_reg
            , ord_no
            , rider_user_id
            , dlvry_rgn1_nm
            , dlvry_rgn2_nm
            , PICK.rgn1_nm AS pick_rgn1_nm
            , PICK.rgn2_nm AS pick_rgn2_nm
            , dlvry.service_type
        FROM dlvry
        LEFT JOIN PICK
        ON dlvry.shop_no = PICK.shop_no
        and dlvry.business_day = PICK.part_date
        WHERE 1=1
        AND dlvry_rgn1_nm = PICK.rgn1_nm
        AND dlvry_rgn2_nm LIKE CONCAT('%', PICK.rgn2_nm, '%')  
        ORDER BY 1,2
    )

    SELECT 
            reg_date 
            , hour_reg 
            , holiday_yn 
            , pick_rgn1_nm 
            , pick_rgn2_nm  
            , count(DISTINCT rider_user_id) AS "라이더수"
            , count(DISTINCT ord_no) AS "주문수"
    FROM dlvry_final
    LEFT JOIN sbbi.dim_date_cd scd 
    ON dlvry_final.reg_date = cast(scd.date_cd AS DATE) 
    WHERE pick_rgn1_nm = '서울특별시'
    GROUP BY 1,2,3,4,5
    ORDER BY 1,2,3,4"""

cursor.execute(sql_query)
results = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]
print(results)

df = pd.DataFrame(results, columns =columns)

cursor.close()
conn.close()

print(df)